# Sentence generation with a given word -- testing different models for English and Polish

Some pre-requisites and things I had to install, mostly written here so I don't forget and that they're all in one place

In [1]:
# !pip install -U torch -q 
# #pytorch 
# #pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 
# #this version with CUDA required for gemma
# !pip install -U transformers datasets -q 
# !pip install -U accelerate -q 
# #required to make gemma work

In [2]:
import torch
torch.cuda.is_available()

True

## English with Gemma 2b (instruct)

In [3]:
#this model is gated, so... logging in to huggingface with #huggingface-cli login# is necessary
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", device_map="auto")

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
input_text = "Write a sentence that contains the word 'dancing'." #...it's the same everytime x_x
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, max_length=100)
print(tokenizer.decode(outputs[0]))

C:\Users\chase\anaconda3\envs\untitled\lib\site-packages\transformers\models\gemma\modeling_gemma.py:573: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


<bos>Write a sentence that contains the word 'dancing'.

The dancing flames illuminated the night sky, casting a mesmerizing glow on the bustling city streets.<eos>


### Sample output: (aka the only output this model will give, no randomness here)

<bos>Write a sentence that contains the word 'dancing'.

The dancing flames illuminated the night sky, casting a mesmerizing glow on the bustling city streets.<eos>

In [5]:
# restart Colab env
exit()

## English with Mistral 7B Instruct v0.2

In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", device_map="auto")

pipe = pipeline(model=model, tokenizer=tokenizer, task='text-generation')

C:\Users\chase\anaconda3\envs\untitled\lib\site-packages\accelerate\utils\modeling.py:1363: UserWarning: Current model requires 905976576 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
human_prompt = "Write me a sentence containing the word 'dancing'."
prompt = f"<s>[INST]{human_prompt}[/INST]"

outputs = pipe(
    prompt,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.9,
    top_p=0.95
)
print(outputs[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
C:\Users\chase\anaconda3\envs\untitled\lib\site-packages\transformers\models\mistral\modeling_mistral.py:688: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


<s>[INST]Write me a sentence containing the word 'dancing'.[/INST] Under the twinkling stars, we all joined hands and danced together, lost in the joyful rhythm of the music.


### Samples:

*(this one took 14 minutes to generate x_x Completely unacceptable...)*

[INST]Write me a sentence containing the word 'dancing'.[/INST] 
- Under the twinkling stars, a group of friends laughed and danced together, creating unforgettable memories at the outdoor summer party.

*(this time it took 2.5 min for some reason, which is not great but fine I guess... except...)*

- Under the twinkling stars, a group of friends let go of their worries and danced joyfully to the rhythm of the lively music.

*(ok what is up with these twinkling stars? x_x)*

- Under the twinkling stars, a group of friends laughed and danced together, creating a joyful and unforgettable moment.

*(changed temperature from 0.1 to 0.5, let's see if it's any different)*

- Under the twinkling fairy lights, a group of friends laughed and cheered as they danced together, lost in the joyful rhythm of the music.

*(HOTTER! temperature 0.9!!)*

- Amidst the lively tunes of the marimba band, everyone was dancing joyfully in the moonlit garden.

*(changed nothing and we're back to the twinkling stars)*

- Under the twinkling stars, a group of children laughed and danced together, their joyful movements illuminating the night.



## English with quantized Mistral 7B Instruct v0.2 GGUF

(https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF)

Current MVP for English, as it's the fastest and the results are good enough

In [ ]:
# !pip3 install huggingface-hub
# !pip install llama-cpp-python
# !huggingface-cli download TheBloke/Mistral-7B-Instruct-v0.2-GGUF mistral-7b-instruct-v0.2.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False

In [3]:
from llama_cpp import Llama

# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
llm = Llama(
    model_path="./mistral-7b-instruct-v0.2.Q4_K_M.gguf",  # Download the model file first
    #chat_format= "llama-2", # The chat format to use - either "llama-1" or "llama-2" -- CAN BE OMITTED FOR DEFAULT FLAVOR
    n_ctx=32768,  # The max sequence length to use - note that longer sequence lengths require much more resources
    n_threads=8,            # The number of CPU threads to use, tailor to your system and the resulting performance
    n_gpu_layers=35         # The number of layers to offload to GPU, if you have GPU acceleration available
)

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from ./mistral-7b-instruct-v0.2.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                

In [4]:
prompt = "Write me a sentence containing the word 'dances'."

# Simple inference example
output = llm(
    f"<s>[INST] {prompt} [/INST]", # Prompt
    max_tokens=120,  # Generate up to 120 tokens
    stop=["</s>"],   # Example stop token - not necessarily correct for this specific model! Please check before using.
    echo=True        # Whether to echo the prompt
)
#print(output)
print(output["choices"][0]["text"])



llama_print_timings:        load time =   20721.89 ms
llama_print_timings:      sample time =       4.94 ms /    23 runs   (    0.21 ms per token,  4660.59 tokens per second)
llama_print_timings: prompt eval time =   20721.67 ms /    20 tokens ( 1036.08 ms per token,     0.97 tokens per second)
llama_print_timings:        eval time =    4640.75 ms /    22 runs   (  210.94 ms per token,     4.74 tokens per second)
llama_print_timings:       total time =   25514.43 ms /    42 tokens


<s>[INST] Write me a sentence containing the word 'dances'. [/INST] At the annual village festival, children and adults alike gather to dance together, celebrating their rich cultural heritage.


### Samples:

[INST]Write me a sentence containing the word 'dancing'.

- Amidst the lively melody, people of all ages were dancing joyfully together at the outdoor festival.
- Amidst the lively tunes of the mariachi band, everyone was dancing and laughing, their joyful spirits intertwined in the warm summer evening.
- Under the twinkling stars, they danced together in perfect harmony, their laughter echoing through the night. *(NOOOOOOOO T-T)*
- As the sun began to set, the couple lost themselves in the rhythm of the music and started dancing passionately under the twinkling stars. *(IT'S BACK, AT THE BACK)*
- As the sun began to set, the crowd at the festival came alive with excitement as they formed a circle and started dancing together, lost in the rhythm of the lively music playing from the speakers.
- As the music played, everyone got up from their seats and started dancing enthusiastically, lost in the joy of the moment.
- Under the twinkling stars, they all gathered around the bonfire, laughing and clapping as they watched the graceful ballerinas perform enchanting dances at the village festival.

*(ok so these are very similar except they take 5 seconds or less, which is obviously strictly better, we'll go with this one after all)*

*(also if we change the word at all, we get different results, so that's good too. Something to keep in mind for sure.)*


[INST] Write me an example sentence containing the word 'dance'. [/INST] 

- On the vibrant and colorful stage, the ballerinas danced gracefully to the enchanting melodies of Tchaikovsky's Swan Lake.
- Sarah twirled gracefully across the floor as she danced to the lively rhythm of the salsa music at the local community center.


[INST] Write me an example sentence containing the word 'dances'. [/INST] 
- At the annual village festival, children of all ages danced joyfully around the maypole, weaving colorful ribbons in a lively and synchronous pattern.
- The colorful performers gracefully danced to the lively rhythm of the traditional music, mesmerizing the crowd with their elegant movements and synchronized steps.


### Polish with Bielik 7B Instruct v0.1 GPTQ (quantized)

Current MVP for Polish, it's fast and does exactly what I need

In [ ]:
%%capture

!pip install optimum -q
!pip install auto-gptq -q
!pip install ctransformers[cuda]

In [1]:
import gc
import torch
import warnings

#warnings.filterwarnings("ignore") # good for demo, not good idea for production :)

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_id = "speakleash/Bielik-7B-Instruct-v0.1-GPTQ" #https://colab.research.google.com/drive/1Al9glPVCuOXbtDsks8cMcuzkuu8YDzpg?usp=sharing#scrollTo=LDo7PEL-gyyy
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=False,
    revision="main"
)

pipe = pipeline(model=model, tokenizer=tokenizer, task='text-generation')
gptq=True

CUDA extension not installed.
CUDA extension not installed.
C:\Users\chase\anaconda3\envs\untitled\lib\site-packages\transformers\modeling_utils.py:4225: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


In [9]:
human_prompt = "Napisz zdanie, w którym znajduje się słowo 'taniec'"
prompt = f"<s>[INST]{human_prompt}[/INST]"

outputs = pipe(
    prompt,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.1, #0.1 is the default, changing it to .5 leads to... German xD
    top_p=0.95
)
print(outputs[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST]Napisz zdanie, w którym znajduje się słowo 'taniec'[/INST]  Taniec to jedna z najstarszych form sztuki, który łączy w sobie elementy muzyki, ruchu i wyrażania emocji.


### Samples:

[INST]Proszę napisać mi zdanie zawierające słowo 'taniec'[/INST]  
- Taniec to jedna z najstarszych form sztuki, która wyraża emocje, historie i kulturę narodów na całym świecie.


[INST]Napisz przykładowe zdanie, w którym się pojawia słowo 'taniec'[/INST]  
- Taniec to jedna z najstarszych form sztuki, która wyraża emocje, energię i kreatywność poprzez ruch i muzykę.


[INST]Napisz przykładowe zdanie, w którym się pojawia słowo **'tańca'**[/INST]  
- Taniec to nie tylko forma rozrywki, ale również sposób na wyrażenie emocji i kreatywności.
- Taniec to nie tylko forma rozrywki, ale również sposób na wyrażenie emocji i kreatywności. *(hmm, usually not THIS repetitive)*


[INST]Napisz zdanie, w którym się pojawia słowo 'tańce'[/INST]  *(temp 0.3)*
- Taniec to jedna z najstarszych form sztuki, która łączy w sobie elementy muzyki, ruchu i wyrażania emocji.


[INST]... 'tańce'[/INST]  *(temp 0.4)*
- Taniec to jedna z najstarszych form sztuki, która pozwala wyrazić emocje i wyzwolić energię.


[INST]... 'tańce'[/INST]  *(temp 0.1)*
- Taniec to jedna z najstarszych form sztuki, która wyraża emocje, energię i kulturę narodów całego świata.


[INST]Napisz zdanie, używając słowo 'taniec'[/INST]  
- Taniec jest jednym z najstarszych i najbardziej uniwersalnych sposobów wyrażania siebie i komunikacji międzyludzkiej.


[INST]Napisz zdanie, w którym ktoś robi 'taniec'[/INST]  
- W sobotni wieczór na parkiecie królował nie tylko taniec, ale i radość życia. #wayyy too vague of an answer


[INST]Napisz zdanie, w którym znajduje się słowo 'taniec'[/INST]  
- Taniec to jedna z najstarszych form sztuki, która wyraża emocje, opowiada historie i łączy ludzi.


*(temperature 0.5 xDDDDD)*
- Fußwork, der die Körpertatsen entfaltt, ist ein Element der Körpertatsen, der auf der Bühne sehr oft genutzt wird.

**Aside from the german one, these are pretty samesy, but honestly really functional. I can try to think of other formats for the prompt that could change the output and continue to test those in the future**

In [6]:
human_prompt = ("Napisz zdanie, w którym pojawia się słowo 'tańczący'.") 
prompt = f"<s>[INST]{human_prompt}[/INST]"

outputs = pipe(
    prompt,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.1,
    top_p=0.95
)
print(outputs[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST]Napisz zdanie, w którym pojawia się słowo 'tańczący'.[/INST]  Wśród tańczących na parkiecie, tylko ona poruszała się z taką gracją i lekkością.


### Samples:

[INST]Napisz zdanie zawierające słowo 'tańczący'.[/INST]  
- W parku można było zobaczyć grupę tańczących par, które spędzały miło czas na świeżym powietrzu.
- W letnie wieczory parki w całym mieście wypełniają się dźwiękami muzyki i radosnymi ludźmi, którzy tańczą, ciesząc się chwilą.
- Wśród tańczących na parkiecie, tylko ona poruszała się z taką gracją i lekkością.

[INST]Napisz zdanie, w którym pojawia się słowo 'tańczący'.[/INST]
- Wśród tańczących na parkiecie, tylko ona poruszała się z taką gracją i lekkością.

**Ok, so some words just produce more varied results regardless, which kinda makes sense actually. Maybe other prompt sentences aren't really necessary, but we'll see how it all comes together.**



In [7]:
human_prompt = ("Napisz 4 zdania, w ktorych się pojawia słowo 'taniec'.")
prompt = f"<s>[INST]{human_prompt}[/INST]"

outputs = pipe(
    prompt,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.1,
    top_p=0.95
)
print(outputs[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST]Napisz 4 zdania, w ktorych się pojawia słowo 'taniec'.[/INST]  1. Taniec to wyjątkowa forma ekspresji, która łączy w sobie ruch, muzykę i emocje.
2. Nauka tańca może być nie tylko przyjemna, ale też bardzo korzystna dla zdrowia.
3. Różne style tańca pozwalają każdemu znaleźć coś, co będzie odpowiadało jego osobowości i preferencjom.
4. Taniec może być sposobem na wyrażenie siebie, budowanie pewności siebie i rozwijanie umiejętności komunikacji.


### Samples:

[INST]Napisz 4 przykładowe zdania, w ktorych znajduje się słowo 'taniec'.[/INST]  Oto przykłady zdań, w których pojawia się słowo "taniec":

1. Taniec to wyrafinowana forma sztuki, która łączy w sobie ruch, muzykę i emocje.
2. Julia pokochała taniec od pierwszego wejrzenia, gdy tylko zobaczyła transmisję z baletu.
3. W weekendowym parku ludzie tańczyli wokół ognia, świętując nadejście wiosny.
4. Taniec towarzyski stał się jego pasją, odkąd zaczął brać lekcje i wygrywać turnieje.

[INST]Napisz 4 przykładowe zdania, w ktorych się pojawia słowo 'taniec'.[/INST]  Oto przykładowe zdania, w których pojawia się słowo "taniec":

1. Taniec to jedna z najstarszych form sztuki, wyrażająca radość i wolność poprzez ruch.
2. Uwielbiam patrzeć, jak moje dzieci tańczą bez tremy, wyrażając siebie w sposób naturalny.
3. Taniec towarzyski to forma sztuki, która łączy partnerów w harmonijnym ruchu i muzyce.
4. Taniec brzucha to technika, która łączy w sobie elementy akrobatyki, izraelskiego folkloru i jogi, wyrażając wewnętrzne piękno kobiecości.

...
1. Taniec to wyrafinowana forma sztuki, która łączy w sobie ruch, muzykę i emocje.
2. Julia kocha taniec i postanowiła zapisać się do szkoły baletowej.
3. W trakcie wesela, goście zostali zachęceni do wspólnego tańca i zabawy.
4. Taniec towarzyski to forma tańca społecznego, która jest wykonywana w parach do różnych rodzajów muzyki.